In [71]:
# %pip install fake_useragent

%pip install selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/04/4d/a6e8afd65b87372e275eb612d564ec68f79195e9b7e27004a3b2cce69686/selenium-4.20.0-py3-none-any.whl.metadata


  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/17/c9/f86f89f14d52f9f2f652ce24cb2f60141a51d087db1563f3fba94ba07346/trio-0.25.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions>=4.9.0 from https://files.pythonhosted.org/packages/01/f3/936e209267d6ef7510322191003885de524fc48d1b43269810cd589ceaf5/typing_extensions-4.11.0-py3-none-any.whl.metadata
  Obtaining dependency information for attrs>=23.2.0 from https://files.pythonhosted.org/packages/e0/44/827b2a91a5816512fcaf3cc4ebc465ccd5d598c45cefa6703fcf4a79018f/attrs-23.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for sortedcontainers from https://files.pythonhosted.org/packages/32/46/9cb0e58b2deb7f82b84065f37f3bffeb12413f947f9

In [197]:
import urllib.request
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import time

Парсим данные вот отсюда https://www.avito.ru/moskva/avtomobili?p=1&radius=0&searchRadius=0


In [ ]:
links = {}

In [179]:
browser = wd.Chrome()
domen = 'https://www.avito.ru'
url = f"{domen}/moskva/avtomobili?p=1&radius=0&searchRadius=0"
browser.get(url)

In [137]:
def get_avto_links():
    car_divs = browser.find_elements(By.CLASS_NAME, 'iva-item-body-KLUuy')
    print(len(car_divs))
    if len(car_divs) == 0:
        # в процессе парсинга приходится пару раз решать высветившийся квиз, 
        # так как айпишник блочится
        time.sleep(60)
        car_divs = browser.find_elements(By.CLASS_NAME, 'iva-item-body-KLUuy')
    for car_div in car_divs:
        div_a = car_div.find_element(By.CLASS_NAME, 'iva-item-title-py3i_')
        # print(div_a)
        title_elem = div_a.find_element(By.TAG_NAME, 'a')
        title = title_elem.get_property('title') if title_elem else None
        car_link = title_elem.get_property('href') if title_elem else None
        # print(title_elem, title, car_link)
        # Extract price
        price_elem = car_div.find_element(By.CLASS_NAME, 'price-price-JP7qe')
        price = price_elem.text.strip() if price_elem else None
        links[title] = {'price': price,
                        'link': car_link}
        # Print or process the extracted details
        print(f"Title: {title}")
        print(f"Price: {price}")
        print(f"Link: {car_link}")

In [139]:
start = 1
end = 100
for page in range(start, end + 1):
    url = f"{domen}/moskva/avtomobili?p={page}&radius=0&searchRadius=0"
    browser.get(url)
    # scroll(browser) – не использую скроллинг, так как иначе сайт виснет и чаще блокирует запросы
    get_avto_links()
browser.quit()


50


Title: Volvo S60 1.6 AMT, 2013, 155 000 км, с пробегом, цена 925 000 руб. - Автомобили в Москве
Price: 925 000 ₽
Link: https://www.avito.ru/moskva/avtomobili/volvo_s60_1.6_amt_2013_155_000_km_3926720552
Title: Hyundai Elantra 1.6 AT, 2017, 98 361 км, с пробегом, цена 899 000 руб. - Автомобили в Москве
Price: 899 000 ₽
Link: https://www.avito.ru/moskva/avtomobili/hyundai_elantra_1.6_at_2017_98_361_km_4137886857
Title: Audi A6 Allroad Quattro 2.5 AT, 2003, 281 228 км, с пробегом, цена 795 000 руб. - Автомобили в Москве
Price: 795 000 ₽
Link: https://www.avito.ru/moskva/avtomobili/audi_a6_allroad_quattro_2.5_at_2003_281_228_km_3987156344
Title: Toyota Avensis 2.0 AT, 2007, 183 290 км, с пробегом, цена 799 000 руб. - Автомобили в Москве
Price: 799 000 ₽
Link: https://www.avito.ru/moskva/avtomobili/toyota_avensis_2.0_at_2007_183_290_km_3899269879
Title: Skoda Rapid 1.6 AT, 2018, 87 178 км, с пробегом, цена 869 000 руб. - Автомобили в Москве
Price: 869 000 ₽
Link: https://www.avito.ru/moskva

In [145]:
len(links) #длина получается намного меньше заявленной в заголовке из-за блокировки страниц и невозможности считать все блоки

1805

In [141]:
# len(links)
df_links = pd.DataFrame(links).T
df_links.index.name = 'title'
df_links.to_csv('links_first_page.csv')

In [182]:
df = pd.read_csv('links_first_page.csv')
df.head()
cars_info = {}

,title,price,link
0,"Kia K900 3.3 AT, 2019, 91 037 км, с пробегом, ...",2 999 000 ₽,https://www.avito.ru/moskva/avtomobili/kia_k90...
1,"Mercedes-Benz V-класс 2.1 AT, 2017, 95 250 км,...",5 650 000 ₽,https://www.avito.ru/moskva/avtomobili/mercede...
2,"Opel Astra GTC 1.4 AT, 2013, 113 000 км, с про...",699 000 ₽,https://www.avito.ru/moskva/avtomobili/opel_as...
3,"Volkswagen Tiguan 2.0 AT, 2015, 165 000 км, с ...",1 195 000 ₽,https://www.avito.ru/moskva/avtomobili/volkswa...
4,"Ford Kuga 1.5 AT, 2018, 103 000 км, с пробегом...",1 780 000 ₽,https://www.avito.ru/moskva/avtomobili/ford_ku...


In [206]:

browser.quit()


In [ ]:
browser = wd.Chrome()
domen = 'https://www.avito.ru'
url = f"{domen}/moskva/avtomobili?p=1&radius=0&searchRadius=0"
browser.get(url)

In [ ]:
# за 70 минут 397 значений=> 1 машина 10,5 секунд => 1800 машин 5,5 часов

In [ ]:
for index, row in df.iterrows():
    if index < 1305:
        continue
    print(index, row)
    browser.get(row['link'])
    # time.sleep(3)
    try:
        characteristics_elem = browser.find_element(By.CLASS_NAME, 'params-paramsList-_awNW')
    except NoSuchElementException:
        cars_info[row['title']] = None
        time.sleep(3)
        continue
    except:
        time.sleep(30)
        characteristics_elem = browser.find_element(By.CLASS_NAME, 'params-paramsList-_awNW')
    chars = characteristics_elem.find_elements(By.CLASS_NAME, 'params-paramsList__item-_2Y2O')
    params = {}
    for char in chars:
        try:
            key, value = char.text.strip().split(': ')
            params[key] = value
        except:
            print(char, 'ERROR')
    if 'Обмен' not in params:
        params['Обмен'] = 0 # обмен не предусмотрен
    try:
        params['Оценка'] = browser.find_element(By.CLASS_NAME, 'desktop-ged5cz').text.strip() # цена ниже / равна / выше оценки Авито
    except:
        params['Оценка'] = None
    # print(params)
    history = browser.find_elements(By.CLASS_NAME, 'list-list-item-pPgHs')
    params['пробег_история'] = None
    params['ДТП'] = None
    params['другое_история'] = None
    params['тех_история'] = None
    params['sold'] = None
    try:
        for item in history:
            item = item.text.strip().split(': ')
            # print(item)
            if len(item) == 1:
                if 'ДТП' in item[0]:
                    params['ДТП'] = item[0]
                else:
                    params['другое_история'] = item[0]
            else:
                # print(item[0], item[1], 'пробег' in item[0])
                if 'пробег' in item[0]:
                    params['пробег_история'] = item[-1] 
                if 'техобслуживан' in item[0]:
                    params['тех_история'] = item[-1] 
                if 'продавался' in item[0]:
                    params['sold'] = item[0] 
    except:
        print('no history')
            
    try:
        geography = browser.find_elements(By.CLASS_NAME, 'style-item-address-georeferences-item-TZsrp')
        metros_info = []
        for geo in geography:
            geo_text = geo.text
            geo_mins = geo.find_element(By.CLASS_NAME, 'style-item-address-georeferences-item-interval-ujKs2')
            metro_name = geo_text[:geo_text.find(geo_mins.text.strip())]
            metros_info.append(f'{metro_name}:{geo_mins.text.strip()}')
        params['Метро'] = ';'.join(metros_info)
    except:
        params['Метро'] = None
    # print(params)
    cars_info[row['title']] = params
    # break    

903 title    Kia Sportage 2.0 AT, 2011, 141 000 км, с пробе...
price                                          1 049 000 ₽
link     https://www.avito.ru/moskva/avtomobili/kia_spo...
Name: 903, dtype: object
<selenium.webdriver.remote.webelement.WebElement (session="0e48e50cf300f630513d9a1fc8b83435", element="f.C27E883886834BC95BC8659DD005621F.d.34848C4EC82434B7CA2F22ED4FF6C8E6.e.61228")> ERROR
<selenium.webdriver.remote.webelement.WebElement (session="0e48e50cf300f630513d9a1fc8b83435", element="f.C27E883886834BC95BC8659DD005621F.d.34848C4EC82434B7CA2F22ED4FF6C8E6.e.61229")> ERROR
<selenium.webdriver.remote.webelement.WebElement (session="0e48e50cf300f630513d9a1fc8b83435", element="f.C27E883886834BC95BC8659DD005621F.d.34848C4EC82434B7CA2F22ED4FF6C8E6.e.61230")> ERROR
<selenium.webdriver.remote.webelement.WebElement (session="0e48e50cf300f630513d9a1fc8b83435", element="f.C27E883886834BC95BC8659DD005621F.d.34848C4EC82434B7CA2F22ED4FF6C8E6.e.61231")> ERROR
<selenium.webdriver.remote.webelem

KeyboardInterrupt: 

In [210]:
len(cars_info)

1306

In [207]:
# len(links)
df_cars = pd.DataFrame(cars_info).T
df_cars.index.name = 'title'
df_cars.to_csv('cars_info.csv')

Общая информация о параметрах:
1. Параметры напрямую относятся к продаваемой машине и описывают техническое составляющее
2. Цель исследования – предсказать цену магины по параметрам
3. Вдальнейшем также можно добавить оценку, число и окраску отзывов (относятся к модели в целом, но возможно также непосредственное влияние на цену)
4. Также для анализа можно взять описание и анализировать его с помощью методов классификации текстов